In [20]:
import pandas as pd
from tabula import read_pdf
import tabula 
import re

#check wether tabula finds Java Runtime Enviroment
tabula.environment_info()

#import and parse data in pdf.format
df = read_pdf("data/Datenananalyse LHM Mietspiegel 2019.pdf", pages='all')

Python version:
    3.7.3 | packaged by conda-forge | (default, Jul  1 2019, 22:01:29) [MSC v.1900 64 bit (AMD64)]
Java version:
    java version "12.0.2" 2019-07-16
Java(TM) SE Runtime Environment (build 12.0.2+10)
Java HotSpot(TM) 64-Bit Server VM (build 12.0.2+10, mixed mode, sharing)
tabula-py version: 1.4.1
platform: Windows-10-10.0.18362-SP0
uname:
    uname_result(system='Windows', node='DaisyDuck', release='10', version='10.0.18362', machine='AMD64', processor='Intel64 Family 6 Model 61 Stepping 4, GenuineIntel')
linux_distribution: ('', '', '')
mac_ver: ('', ('', '', ''), '')
    


In [21]:
#create dictionary with number for every district
munich_dict = {
    "2" : "Ludwigsvorstadt-Isarvorstadt",  
    "3" : "Maxvorstadt",                                               
    "4" : "Schwabing-West",
    "9" :"Neuhausen-Nymphenburg",
    "5" : "Au-Haidhausen",                                              
    "12" : "Schwabing-Freimann",                                         
    "13" : "Bogenhausen",                                                
    "11" : "Milbertshofen-Am Hart",                                      
    "6" : "Sendling",                                                   
    "8" : "Schwanthalerhöhe",                                           
    "16" : "Ramersdorf-Perlach",                                         
    "7" : "Sendling-Westpark",                                          
    "19" : "Thalkirchen-Obersendling-Forstenried-Fürstenried-Solln",     
    "25" : "Laim",                                                       
    "17" : "Obergiesing",                                                
    "18" : "Untergiesing-Harlaching",                                    
    "1" : "Altstadt-Lehel",                                             
    "15" : "Tudering-Riem",                                              
    "21" : "Pasing-Obermenzing",                                         
    "14" : "Berg am Laim",                                               
    "10" : "Moosach",                                                    
    "20" : "Hadern",                                                     
    "24" : "Feldmoching-Hasenbergl",                                      
    "22" : "Aubing-Lochhausen-Langwied",                                  
    "23" : "Allach-Untermenzing",                                         
}

In [22]:
#drop rows where, at the beginning of every page, the column name was registered as a value
df = df[df.Stadtbezirksteil != 'Stadtbezirksteil']

#convert variable rent to float
df.loc[:,'Nettokaltmiete\rpro m2 in Euro'] = df.loc[:,'Nettokaltmiete\rpro m2 in Euro'].str.replace(',', '.').astype(float)

In [23]:
# create new column for neighbourhood name
df['neighbourhood'] = ""

#iterate over df and dictionary and assign neighbourhood name to every case
for index, row in df.iterrows():
    district_number = re.search("(\d+)\.", row.Stadtbezirksteil) #extract district number from string using regular expressions
    for key in munich_dict.keys():
        if (key == district_number.group(1)):
            df.loc[index, 'neighbourhood'] = munich_dict[key]

In [24]:
df['neighbourhood'].describe()

count                      3024
unique                       25
top       Neuhausen-Nymphenburg
freq                        235
Name: neighbourhood, dtype: object

In [25]:
df['average_rent_euro'] = df['Nettokaltmiete\rpro m2 in Euro']
df = df.drop(columns = 'Nettokaltmiete\rpro m2 in Euro')
df.describe()

,average_rent_euro
count,3024.000000
mean,11.724312
std,3.135560
min,2.880000
25%,9.707500
50%,11.740000
75%,13.640000
max,27.900000


In [27]:
#group df by neighbourhoods and calculate mean for every neighbourhood
final = df.groupby('neighbourhood').mean()

#export generated data to csv-file
final.to_csv('data/rent_index_Munich.csv')